In [1]:
import pandas as pd
import numpy as np
from pyprojroot import here

from spellchecker import SpellChecker
from language_tool_python import LanguageTool

In [2]:
train_df = pd.read_csv(here('commonlit-evaluate-student-summaries/summaries_train.csv'))
train_promts_df = pd.read_csv(here('commonlit-evaluate-student-summaries/prompts_train.csv'))

In [3]:
spell_checker = SpellChecker()



In [4]:
test = train_df['text']

In [5]:
for text in test:
    # slip text into words
    words = text.split()
    misspelled = []
    for word in words:
        # remove punctuation and symbols from word
        word = word.strip('.,;:–-—!?%()[]¨0123456789\'\"')
        if word == '' or word == ' ' or len(word) < 3:
            pass
        else:
            word_check = spell_checker.unknown([word])
            if len(word_check) > 0:
                misspelled.append(word)
                

In [6]:
tool = LanguageTool('en-US')

test = train_df['text'][0:1]

matches = tool.check(test)

for match in matches:
    print(match)
    
tool.close()

ModuleNotFoundError: No java install detected. Please install java to use language-tool-python.